In [2]:
import pandas as pd

In [3]:
df1 = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet")

In [4]:
df1.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [5]:
!pip install scikit-learn

In [6]:
import sklearn

In [7]:
len(df1.columns)

19

In [8]:
df1['duration'] = (df1['tpep_dropoff_datetime'] - df1['tpep_pickup_datetime']).dt.total_seconds()/ 60

In [9]:
df1.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,8.433333
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,6.316667
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,12.750000
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,9.616667
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,10.833333


In [10]:
df1['duration'].std()

np.float64(42.59435124195458)

In [11]:
#drop outliers
df1_drop = df1[(df1['duration'] >= 1) & (df1['duration'] <= 60)]
fraction = len(df1_drop)/len(df1)
print (fraction * 100)

98.1220282212598


In [12]:
#one-hot encoding
from sklearn.feature_extraction import DictVectorizer

In [13]:
df1_drop.loc[:,'PULocationID'] = df1_drop['PULocationID'].astype(str)
df1_drop.loc[:,'DOLocationID'] = df1_drop['DOLocationID'].astype(str)
conv_dict = df1_drop[['PULocationID','DOLocationID']].to_dict(orient = 'records')

/tmp/ipykernel_27688/2027061861.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['161' '43' '48' ... '114' '230' '262']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df1_drop.loc[:,'PULocationID'] = df1_drop['PULocationID'].astype(str)
/tmp/ipykernel_27688/2027061861.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['141' '237' '238' ... '239' '79' '143']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df1_drop.loc[:,'DOLocationID'] = df1_drop['DOLocationID'].astype(str)


In [14]:
#Initializing Dictionary Vectorizer
div = DictVectorizer()

In [23]:
x_train = div.fit_transform(conv_dict)
print(x_train.shape[1])

515


In [18]:
#training model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error

In [29]:
model = LinearRegression()
model.fit(x_train, df1_drop['duration'])

#RMSE
y_pred = model.predict(x_train)
rmse_pred = root_mean_squared_error(df1_drop['duration'], y_pred)
print(rmse_pred)

7.649261932106969


In [24]:
df2 = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-02.parquet')


In [25]:
df2.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2,2024-02-01 00:04:45,2024-02-01 00:19:58,1.0,4.39,1.0,N,68,236,1,20.5,1.0,0.5,1.28,0.00,1.0,26.78,2.5,0.00
1,2,2024-02-01 00:56:31,2024-02-01 01:10:53,1.0,7.71,1.0,N,48,243,1,31.0,1.0,0.5,9.00,0.00,1.0,45.00,2.5,0.00
2,2,2024-02-01 00:07:50,2024-02-01 00:43:12,2.0,28.69,2.0,N,132,261,2,70.0,0.0,0.5,0.00,6.94,1.0,82.69,2.5,1.75
3,1,2024-02-01 00:01:49,2024-02-01 00:10:47,1.0,1.10,1.0,N,161,163,1,9.3,3.5,0.5,2.85,0.00,1.0,17.15,2.5,0.00
4,1,2024-02-01 00:37:35,2024-02-01 00:51:15,1.0,2.60,1.0,N,246,79,2,15.6,3.5,0.5,0.00,0.00,1.0,20.60,2.5,0.00


In [26]:
df2['duration'] = (df2['tpep_dropoff_datetime'] - df2['tpep_pickup_datetime']).dt.total_seconds()/ 60

In [36]:
#drop outliers
df2_drop = df2[(df1['duration'] >= 1) & (df2['duration'] <= 60)]

/tmp/ipykernel_27688/821433943.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df2_drop = df2[(df1['duration'] >= 1) & (df2['duration'] <= 60)]


In [34]:
df2_drop.loc[:,'PULocationID'] = df2_drop['PULocationID'].astype(str)
df2_drop.loc[:,'DOLocationID'] = df2_drop['DOLocationID'].astype(str)
val_dict = df2_drop[['PULocationID','DOLocationID']].to_dict(orient = 'records')

In [35]:
val_x = div.transform(val_dict)

val_y_pred = model.predict(val_x)
rmse_val = root_mean_squared_error(df2_drop['duration'], val_y_pred)
print(rmse_val)

8.454048217732586
